#  1. Thomson data 파악 

In [9]:
import pandas as pd
import zipfile
import csv
import os

os.chdir("E:/patent/all_patent")

zf_patent1 = zipfile.ZipFile('thomson_harvard_matched.zip') 
csv_patent1 = zf_patent1.open('thomson_harvard_matched.csv')
csv_patent_reader1 = csv.reader(csv_patent1)

### Where is info about class?
어떤 colummn이 class 정보를 담고 있는지 파악  
일단 첫 번째 row의 value부터 살펴보자

In [10]:
header = csv_patent_reader1.next()
header

['-i0t0',
 'Age at Financing (Months)-i0t0',
 'Company City-i0t0',
 'Company Current Operating Stage-i0t0',
 'Company Current Public Status-i0t0',
 'Company Founded Year-i0t0',
 'Company ID-i0t0',
 'Company IPO Date-i0t0',
 'Company Investment Stage 1 at Round Date-i0t0',
 'Company Investment Stage 2 at Round Date-i0t0',
 'Company Investment Stage 3 at Round Date-i0t0',
 'Company MoneyTree Region-i0t0',
 'Company Name-i0t0',
 'Company Nation-i0t0',
 'Company Number of Employees-i0t0',
 'Company State / Region-i0t0',
 'Company Status-i0t0',
 'Company VE Primary Industry Class-i0t0',
 'Company VE Primary Industry Major Group-i0t0',
 'Company World Location (Branch Office)-i0t0',
 'Company Zip Code-i0t0',
 'Cumulative Dividends-i0t0',
 'Deal Value (USD Mil)-i0t0',
 'Debt Amount (USD Mil)-i0t0',
 'Equity Amount Disclosed (USD Mil)-i0t0',
 'Equity Amount Estimated (USD Mil)-i0t0',
 'Firm - First Investment Date-i0t0',
 'Firm - Last Investment Date-i0t0',
 'Firm - Total Number of Deals-i0t0'

In [3]:
row1 = csv_patent_reader1.next()
for i in range(0,len(row1)):
    if row1[i] != '':
        print i, header[i], row1[i]

??? 왜 자꾸 row에 아무 것도 없지? 값이 있는 첫 번째 row 뽑아보자

In [3]:
tmp=0
row1=[]
for line in csv_patent_reader1:
    if tmp==0:
        for i in range(0,len(header)):
            if line[i] != '':
                print i, header[i], line[i]
                tmp = tmp+1
    else:
        break  

0 -i0t0 493155
1 Age at Financing (Months)-i0t0 474
2 Company City-i0t0 El Segundo
3 Company Current Operating Stage-i0t0 -
4 Company Current Public Status-i0t0 Public
5 Company Founded Year-i0t0 1945
6 Company ID-i0t0 C901930294
7 Company IPO Date-i0t0 1997-03-21 00:00:00
8 Company Investment Stage 1 at Round Date-i0t0 Other
9 Company Investment Stage 2 at Round Date-i0t0 Public Market
10 Company Investment Stage 3 at Round Date-i0t0 Open Market Purchase
11 Company MoneyTree Region-i0t0 LA/Orange County
12 Company Name-i0t0 Mattel, Inc.
13 Company Nation-i0t0 United States
14 Company Number of Employees-i0t0 29000
15 Company State / Region-i0t0 California
16 Company Status-i0t0 Went Public
17 Company VE Primary Industry Class-i0t0 Non-High Technology
18 Company VE Primary Industry Major Group-i0t0 Non-High-Technology
19 Company World Location (Branch Office)-i0t0 -
20 Company Zip Code-i0t0 90245
21 Cumulative Dividends-i0t0 -
22 Deal Value (USD Mil)-i0t0 -
23 Debt Amount (USD Mil)-i0t

### 기본 정보
company 1개, investment date 순으로 firm 나열  
Equity Amount Disclosed : 펀딩 금액  
Firm Total Estimated Equity Invested in Company (USD Mil) : firm의 funding 총액 to company
investment_date : 펀딩 날짜  
round_number : 라운드  
valuation_at_transaction_date_us : 당시 밸류에이션 (missing 많음)  


### problem
1. 같은 날에 투자한 firm 각각이 얼마나 투자했는지 알 수 없음
2. Company name != Assignee org
 - case 1 : company name 동일, assignee 이름 다름
 - case 2 : company name 다름 or nan, assingee 이름 다름

# 2. Problem 대처  
### 근데 그게 중요한가?

deal 당 funding 금액을 몰라도  
1. firm의 funding 횟수 / 여부  
2. funding 총액  

교수님께 확인 필요

중요함... deal date, deal 당 금액 정보 필요

### 1. firm 투자 금액 확인 - facebook으로 비교

In [7]:
csv_patent_reader1.next()
fb=[]
for line in csv_patent_reader1:
    if 'acebook' in line[12]:
        fb.append(line)
        break

fb=fb[0]

In [18]:
myfile = open("fb.csv", 'wb')
myfile_writer = csv.writer(myfile,quoting=csv.QUOTE_ALL)
myfile_writer.writerow(header)
myfile_writer.writerow(fb)
myfile.close()

In [12]:
import pandas as pd
import zipfile
import csv
import os

os.chdir("E:/patent/all_patent")
fb_all=pd.read_csv('fb.csv')

In [97]:
sel_info=[12, 25, 36, 42, 51, 64, 69]
fb=fb_all.ix[:,sel_info]
fb

,Company Name-i0t0,Equity Amount Estimated (USD Mil)-i0t0,Firm Name-i0t0,Firm Total Estimated Equity Invested in Company (USD Mil)-i0t0,Fund Estimated Equity Invested in Company at Investment Date (USD Mil)-i0t0,Fund Total Estimated Equity Invested in Company (USD Mil)-i0t0,Investment Date-i0t0
0,Facebook Inc,12.4,Accel Partners & Co Inc,19.0375,12.4,19.0375,2005-05-11


In [98]:
for i in range(1,40):
    sel_info = [x+94 for x in sel_info]
    tmp=fb_all.ix[0,sel_info]
    tmp=tmp.tolist()
    if pd.isnull(tmp[0])==True:        
        break
    else:
        fb.loc[i]=tmp
        
fb.to_csv('fb_info.csv',index=False)
fb

C:\Users\DW\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Company Name-i0t0,Equity Amount Estimated (USD Mil)-i0t0,Firm Name-i0t0,Firm Total Estimated Equity Invested in Company (USD Mil)-i0t0,Fund Estimated Equity Invested in Company at Investment Date (USD Mil)-i0t0,Fund Total Estimated Equity Invested in Company (USD Mil)-i0t0,Investment Date-i0t0
0,Facebook Inc,12.4,Accel Partners & Co Inc,19.0375,12.4,19.0375,2005-05-11
1,Facebook Inc,27.5,Accel Partners & Co Inc,19.0375,6.6375,19.0375,2006-06-24
2,Facebook Inc,27.5,Greylock Partners LLC,13.531,6.6375,13.531,2006-06-24
3,Facebook Inc,27.5,"Founders Fund, The",0.95,0.95,0.95,2006-06-24
4,Facebook Inc,27.5,Undisclosed Firm,6.6375,6.6375,6.6375,2006-06-24
5,Facebook Inc,27.5,Meritech Capital Partners,6.6375,6.6375,6.6375,2006-06-24
6,Facebook Inc,60,Undisclosed Firm,60,60,60,2007-11-30
7,Facebook Inc,-,Global Founders Capital Management GmbH,15,15,15,2008-01-15
8,Facebook Inc,15,TriplePoint Capital LLC,-,-,-,2008-05-09
9,Facebook Inc,200.999,Millennium TVP Management Company LLC,250,200,250,2009-05-26


### 2. patent assignee, invested company 비교

In [1]:
#basic settings
import pandas as pd
import zipfile
import csv
import os

os.chdir("E:/patent/all_patent")

zf_patent1 = zipfile.ZipFile('harvard_dataset_final_merged_output_1.zip') 
csv_patent1 = zf_patent1.open('harvard_dataset_final_merged_output_1.csv')

csv_patent_reader1 = csv.reader(csv_patent1)

zf_patent2 = zipfile.ZipFile('harvard_dataset_final_merged_output_2.zip') 
csv_patent2 = zf_patent2.open('harvard_dataset_final_merged_output_2.csv')
csv_patent_reader2 = csv.reader(csv_patent2)

zf_thomeson = zipfile.ZipFile('thomson_harvard_matched.zip') 
csv_thomeson = zf_thomeson.open('thomson_harvard_matched.csv')
csv_thomeson_reader = csv.reader(csv_thomeson)

patent에서 원하는 정보 담긴 column만 뽑자  
원하는 정보 = patent id, org

In [49]:
col_info_patent = pd.read_csv('harvard_dataset_final_merged_columns.csv',index_col = 0)
col_info_patent['selector']=False
info=[19493,19499]
col_info_patent.iloc[info,1]=True
print col_info_patent.iloc[info,]

          column_name selector
19494     patent_id_0     True
19500  organization_0     True


In [50]:
selection_patent = [False] + col_info_patent.selector.tolist()
print selection_patent[:10]

[False, False, False, False, False, False, False, False, False, False]


investment에서 원하는 정보 담긴 column만 뽑자  
원하는 정보 = company name

In [51]:
col_info_thomeson = pd.read_csv('thomson_harvard_matched_column_formatted.csv',index_col = 0)
col_info_thomeson['selector']=False
col_info_thomeson.iloc[12,1]=True
print col_info_thomeson.iloc[12,]

column_name    Company Name-i0t0
selector                    True
Name: 13, dtype: object


In [55]:
selection_thomeson = col_info_thomeson.selector.tolist()
print selection_thomeson[:14]

[False, False, False, False, False, False, False, False, False, False, False, False, True, False]


patent id, assignee, company로 된 file 생성  
중간에 끊음 (너무 오래 걸려)

In [56]:
from itertools import compress

myfile = open("patent_thomeson_selected_columns_extracted.csv", 'wb')
myfile_writer = csv.writer(myfile,quoting=csv.QUOTE_ALL)

i = 0
num_total_row = 5271460

for line in csv_thomeson_reader:
    
    if i <= int(num_total_row / 2) + 1:
        harvard_data = next(csv_patent_reader1)
    else:
        if i == int(num_total_row / 2) + 2:
            discard_column_line_from = next(csv_patent_reader2)
            
        harvard_data = next(csv_patent_reader2)
    
    sumline = list(compress(harvard_data, selection_patent)) + list(compress(line, selection_thomeson))
    myfile_writer.writerow(sumline)
    
    i = i+1

myfile.close()

KeyboardInterrupt: 

In [59]:
i

2093222

### Case 1 검증

In [75]:
extract=pd.read_csv('patent_thomeson_selected_columns_extracted.csv')
extract.iloc[550:600,]

,patent_id_0,organization_0,Company Name-i0t0
550,3858792.0,"COMPAK SYSTEMS, INC.",NaN
551,3858793.0,"DONALDSON COMPANY, INC.",NaN
552,3858794.0,ALFA-LAVAL AB,NaN
553,3858795.0,INTERNATIONAL EQUIPMENT COMPANY,NaN
554,3858796.0,NaN,NaN
555,3858797.0,NaN,NaN
556,3858798.0,NaN,NaN
557,3858799.0,"RICOH COMPANY, LTD.",NaN
558,3858800.0,STANDARD THOMSON CORPORATION,NaN
559,3858801.0,STANDARD THOMSON CORPORATION,NaN


In [77]:
match=[110,145,209,295,353,467,480,487,501,577]
extract.iloc[match,]

,patent_id_0,organization_0,Company Name-i0t0
110,3858351.0,MATTEL INC.,"Mattel, Inc."
145,3858386.0,FIBER INDUSTRIES INC.,Fiber Industries Inc.
209,3858451.0,SINGER COMPANY,Singer Company
295,3858537.0,SINGER COMPANY,Singer Company
353,3858595.0,"CHAMPION INDUSTRIES, INC.",Champion Industries Inc
467,3858709.0,OWENS-ILLINOIS INC.,Owens-Illinois Inc
480,3858722.0,SINGER COMPANY,Singer Company
487,3858729.0,"AMSTED INDUSTRIES, INCORPORATED",Amsted Industries Inc
501,3858743.0,OWENS-ILLINOIS INC.,Owens-Illinois Inc
577,3858819.0,BUTLER AUTOMATIC INC.,Butler Automatic Inc


역시 예상대로 GM같은 곳은 match 불가(투자 받은 적 없음)  
그렇다면 일단 무작위로 matched sample만 뽑아보자

In [153]:
ext_10000 = extract.iloc[10000:300000,]
check=ext_10000.loc[pd.isnull(ext_10000['Company Name-i0t0'])!=True]
check=check.sort_values(['Company Name-i0t0'])
check.to_csv('test.csv',index=False)

In [154]:
len(check.index)

6258

In [156]:
check=pd.read_csv('test.csv',)
check.iloc[0,2] == check.iloc[1,2] and check.iloc[0,1] != check.iloc[1,1]

False

In [169]:
for i in range(0,len(check)-1):
    if check.iloc[i,2] == check.iloc[i+1,2] and check.iloc[i,1] != check.iloc[i+1,1]:
        print check.iloc[i:i+2,]

   patent_id_0 organization_0 Company Name-i0t0
2      4039325      AMAX INC.        AMAX, Inc.
3      4093526              s        AMAX, Inc.
   patent_id_0 organization_0 Company Name-i0t0
3      4093526              s        AMAX, Inc.
4      4072507             df        AMAX, Inc.
   patent_id_0 organization_0 Company Name-i0t0
4      4072507             df        AMAX, Inc.
5      4072543      AMAX INC.        AMAX, Inc.
   patent_id_0 organization_0 Company Name-i0t0
5      4072543      AMAX INC.        AMAX, Inc.
6      3933975              v        AMAX, Inc.
   patent_id_0 organization_0 Company Name-i0t0
6      3933975              v        AMAX, Inc.
7      4012484      AMAX INC.        AMAX, Inc.
   patent_id_0 organization_0 Company Name-i0t0
7      4012484      AMAX INC.        AMAX, Inc.
8      3933976              v        AMAX, Inc.
   patent_id_0 organization_0 Company Name-i0t0
8      3933976              v        AMAX, Inc.
9      4104359      AMAX INC.        AMA

### 정리

1. Assignee 이름 이슈
 - 이름별로 정렬된 건 아님
 - Assignee 이름은 noise 많음
 - 그러나 noise 많은 assignee 이름을 company name과 성공적으로 match
 - 따라서 company name 기준으로 묶으면 firm level 분석 가능!  
2. 미해결 문제
 - 100% match 라고 장담 불가능 (case 2 error에 대해서 검증 불가)
 - company name이 없는 assignee (투자 받은 적 없는 assignee)는 firm level 분석 불가